# Working with the en_IN locale

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/enabling-languages/python-i18n/blob/main/colab/en_IN_on_colab.ipynb)

The *en_IN.UTF-8* locale is available on linux, but absent on macOS, the locale allows the correct formatting and display of numbers and currency according to Indian conventions, including the (3, 2) pattern for grouping separators.

_The notebook contains example code that will try to set the locale to **en_IN.UTF-8**, and if the locale is unavailable, the code will install the locale for this notebook, and to stop the runtime environment._

**_It will be necessary to manually restart the runtime environment and rerun the notebook._**

## Setup

In [34]:
import locale
loc = "en_IN.UTF-8"

try:
  import google.colab
  IN_COLAB = True
except ImportError:
  IN_COLAB = False
if IN_COLAB:
  !pip install PyICU
  try:
    locale.setlocale(locale.LC_ALL, loc)
  except locale.Error:
    print("Currently installing required locale. After the install the runtime environment will restart, please rerun all code.")
    !sudo locale-gen {loc}
    !sudo update-locale
    import os
    os.kill(os.getpid(), 9)
else:
  locale.setlocale(locale.LC_ALL, loc)

from icu import Locale, NumberFormatter, CurrencyUnit, RuleBasedNumberFormat, URBNFRuleSetTag, NumberFormat

## Using the locale module

`locale.currency()` will convert a normalised (unlocalised) numeral into a formatted numeral string based on the conventions of the current locale. By default the conversion does not include grouping (thousands) separators in the formatted string. To enable grouping seperators, set the `grouping` parameter to `True`. The default value is `False`.

In [35]:
lst = [2.54, 4.0, 3, 9.95, 5.4]
print([locale.currency(i, grouping=True) for i in lst])

['₹ 2.54', '₹ 4.00', '₹ 3.00', '₹ 9.95', '₹ 5.40']


In [36]:
lst2 = [123452.54, 4367843.0, 120030, 900020.95, 5000000.0]
print([locale.currency(i, grouping=True) for i in lst2])


['₹ 1,23,452.54', '₹ 43,67,843.00', '₹ 1,20,030.00', '₹ 9,00,020.95', '₹ 50,00,000.00']


The _glibc_ based _en_IN_ locale does not insert a space between the currency symbol and the numerals. There is an internal function intended for testing, `locale._override_localeconv`, that can change some of the locale settings, in theory it should be reserved for testing only.

In [37]:
locale._override_localeconv["p_sep_by_space"] = 1
print([locale.currency(i, grouping=True) for i in lst2])


['₹ 1,23,452.54', '₹ 43,67,843.00', '₹ 1,20,030.00', '₹ 9,00,020.95', '₹ 50,00,000.00']


## Using PyICU

There are a number of ways of creating a formatter instance for currency. The original method was `icu.NumberFormat`, the newer method is `icu.NumberFormatter`, which superceeded `icu.NumberFormat`.

Using `icu.NumberFormatter`:

In [38]:
icu_loc = Locale.createCanonical(loc)
formatter = NumberFormatter.with_().unit(CurrencyUnit('INR')).locale(icu_loc)
print([str(formatter.formatDouble(i)).replace('\xa0', ' ') for i in lst2])


['₹ 1,23,452.54', '₹ 43,67,843.00', '₹ 1,20,030.00', '₹ 9,00,020.95', '₹ 50,00,000.00']


Using `icu.RuleBasedNumberFormat
`:

In [47]:
rbnf = RuleBasedNumberFormat.createCurrencyInstance(icu_loc)
print(rbnf.format(lst2[0]))

₹ 1,23,452.54


Using `icu.NumberFormat`:

In [40]:
nf_formatter = NumberFormat.createCurrencyInstance(icu_loc)
print(nf_formatter.format(lst2[0]))

₹ 1,23,452.54


### Text version of numbers via SPELLOUT

In [39]:
rb_formatter = RuleBasedNumberFormat(URBNFRuleSetTag.SPELLOUT, icu_loc)
print(rb_formatter.format(lst2[0]))

one lakh twenty-three thousand four hundred fifty-two point five four
